In [1]:
!pip install wandb

In [2]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [11]:
from transformers import T5Tokenizer, T5ForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import Dataset

from sklearn.model_selection import train_test_split

import pandas as pd
import torch

##### Custom Dataset 정의

In [17]:
class CustomDataset(Dataset):
    def __init__(self, tokenizer, texts, labels, max_length=512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.labels = labels
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        inputs = self.tokenizer(
            self.texts[idx],
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
            )
        inputs['input_ids'] = inputs['input_ids'].squeeze()
        inputs['attention_mask'] = inputs['attention_mask'].squeeze()
        inputs['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return inputs

##### 데이터 불러오기 및 Train, Validation set으로 쪼개기

In [18]:
final_data_df = pd.read_csv('/content/drive/MyDrive/BOAZ/mini_project_2/new_final.csv')

CoT_sentences = final_data_df['prompt'].tolist() # CoT문장들 리스트
target_scores = final_data_df['target_rating'].tolist() # 목표평점 리스트

print(f'CoT 문장 개수 : {len(CoT_sentences)}')
print(f'목표평점 개수 : {len(target_scores)}')


# 데이터셋 분리
train_sentences, valid_sentences, train_scores, valid_scores = train_test_split(CoT_sentences,
                                                                                target_scores,
                                                                                test_size=0.2,
                                                                                random_state=42)

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForSequenceClassification.from_pretrained("google/flan-t5-small", num_labels=2)


train_dataset = CustomDataset(tokenizer,
                              train_sentences,
                              train_scores,
                              max_length=256)

valid_dataset = CustomDataset(tokenizer,
                              valid_sentences,
                              valid_scores,
                              max_length=256)

CoT 문장 개수 : 4000
목표평점 개수 : 4000


Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google/flan-t5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##### 평가 지표 정의(accuracy, f1)

In [24]:
def compute_metrics(pred):
    labels = pred.label_ids
    predictions = pred.predictions[0] if isinstance(pred.predictions, tuple) else pred.predictions
    preds = predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")  # F1-score
    return {"accuracy": acc, "f1": f1}

##### Training Arguments 정의

In [27]:
training_args = TrainingArguments(
    output_dir='./result',
    overwrite_output_dir=True,
    logging_dir='./logs',
    logging_steps=10,
    num_train_epochs=30,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    save_strategy='epoch',
    eval_strategy='epoch',
    load_best_model_at_end=True, # 최적의 모델 로드
    metric_for_best_model="eval_loss", # Early Stopping 기준 메트릭
    greater_is_better=False,     # 평가 메트릭이 작을수록 더 나은 경우 (e.g., loss)
    save_total_limit=1,
    learning_rate=1e-5,
    weight_decay=0.01,
    warmup_steps=100,
    report_to='wandb',
    run_name='kau_chatbot_recsys_flan_t5_regression',
    seed=42
)

##### 학습

In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()
trainer.evaluate()

model.save_pretrained("/content/drive/MyDrive/BOAZ/mini_project_2/kau_chatbot_recsys_flan_t5_regression")
tokenizer.save_pretrained("/content/drive/MyDrive/BOAZ/mini_project_2/kau_chatbot_recsys_flan_t5_regression")

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.100500,0.150504,0.955000,0.954972
2,0.096100,0.153421,0.955000,0.954977


KeyboardInterrupt: 